In [1]:
import matplotlib
matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os
import umap
import parmap

import glob2
import numpy as np
from sklearn.decomposition import PCA

from shapely.geometry import Polygon
from shapely import geometry
from matplotlib.patches import Polygon
from scipy.spatial import ConvexHull


# visualize results module
from Visualize import *
from Visualize import PCA_Analysis
import pickle as pk

# 
#data_dir = '/media/cat/4TBSSD/yuki/'
data_dir = '/media/cat/4TBSSD/yuki/'


Autosaving every 180 seconds


In [22]:
###############################################################
################# VISUALIZE PCA 2D MAPS #######################
###############################################################

# ***************** Fig 2B *********************

masktxt = np.int32(np.loadtxt('/media/cat/4TBSSD/yuki/IJ2/genericmask.txt'))
#print (masktxt)

mask = np.ones((128,128),'float32')
for k in range(masktxt.shape[0]):
    #print (masktxt[k][0],masktxt[k][1])
    mask[masktxt[k][0],masktxt[k][1]]=np.nan

# 
fig=plt.figure()
data = np.load('/media/cat/4TBSSD/yuki/IJ2/tif_files/IJ2pm_Feb5_30Hz/IJ2pm_Feb5_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95_spatial.npy')
#print (data.shape)
data = data.reshape(data.shape[0],128,128)
for k in range(4):
    ax=plt.subplot(2,2,k+1)
    vmin = np.min(data[k])
    vmax = np.max(data[k])
    
    # 
    #print (k, vmin,vmax)
    plt.imshow(data[k]*mask, vmin=vmin, vmax=vmax)
    plt.xticks([])
    plt.yticks([])
plt.show()

In [2]:
####################################################
#### COMPUTE PCA AND UMAP AVERAGE SCATER PLOT ######
####################################################

# ******************** FIG 2D ***********************

data = np.load('/home/cat/concatenated_trials.npy')
#data = np.load('/media/cat/4TBSSD/yuki/IJ2/tif_files/IJ2pm_Mar2_30Hz/IJ2pm_Mar2_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy')
#data = np.load('/media/cat/4TBSSD/yuki/IJ2/tif_files/IJ2pm_Mar3_30Hz/IJ2pm_Mar3_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy')
data = np.load('/media/cat/4TBSSD/yuki/IJ1/tif_files/IJ1pm_Mar2_30Hz/IJ1pm_Mar2_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy')
print ("raw data: ", data.shape)
data_all = data[:,:,:900]
data_all = data_all[:,:4,750:]
mean = data_all.mean(0)

print ("data_all: ", data_all.shape)
print ("Mean: ", mean.shape)
# 
data_in = []
mean2 = []
width = 5
for k in range(0, data_all.shape[2],width):
    data_in.append(data_all[:,:,k:k+width])
    mean2.append(mean[:,k:k+width])

mean2 = np.array(mean2)
data_in = np.array(data_in) 
print ("data in: ", data_in.shape)
print ("mean: ", mean2.shape)

#
mean2 = mean2.reshape(mean2.shape[0],-1)[:,None]
print ("mean: ", mean2.shape)


# 
data_in = data_in.reshape(data_in.shape[0], data_in.shape[1],-1)
print ("data in: ", data_in.shape)

d = []
for k in range(data_in.shape[0]):
    d.extend(data_in[k])

d.extend(mean2[:,0])
data_in = np.array(d)
print ("data in: ", data_in.shape)
# 


# 
X = data_in
print ("data input: ", X.shape)


#############################################
#############################################
#############################################
#
pca = PCA(n_components=2)
pca.fit(X)
p = pca.transform(X)
print ("Pca output: ", p.shape)

import umap
fit = umap.UMAP()
%time u = fit.fit_transform(X)
print ("Umap output: ", u.shape)


#######################################
########### PLOT RESULTS ##############
#######################################
# flatten data for PCA
t_flat = []

n_chunks = 30              # MANUAL SETTINGS
chunk_width = 10
n_trials = 62
cs = []
for k in range(n_chunks):
    cs.append(k*np.ones(n_trials)/n_chunks)
cs=np.hstack(cs)
#print (cs[:300])
    
clrs = plt.cm.viridis(cs)
print (clrs)

print("p :", p.shape)

# 
fig = plt.figure(figsize=(10,10))
cm = plt.cm.get_cmap('viridis')
ax=plt.subplot(211)

plt.scatter(p[:,0], p[:,1], c=clrs, s=25, edgecolor = 'black', alpha=.5)

#
plt.scatter(p[-n_chunks:,0], p[-n_chunks:,1], 
            c=np.arange(n_chunks), 
            s=150, cmap=plt.cm.viridis,
            edgecolor = 'black', alpha=.8)
    

ax=plt.subplot(212)
sc = plt.scatter(u[:,0], u[:,1], c=clrs, s=25, edgecolor = 'black', alpha=.5)
plt.scatter(u[-n_chunks:,0], u[-n_chunks:,1], 
            c=np.arange(n_chunks), 
            s=150, cmap=plt.cm.viridis,
            edgecolor = 'black', alpha=.8)


if False:
    plt.savefig('/home/cat/pca_single_trials.png',dpi=300)
    plt.close()
else:
    plt.show()
    

raw data:  (61, 7, 1801)
data_all:  (61, 4, 150)
Mean:  (4, 150)
data in:  (30, 61, 4, 5)
mean:  (30, 4, 5)
mean:  (30, 1, 20)
data in:  (30, 61, 20)
data in:  (1860, 20)
data input:  (1860, 20)
Pca output:  (1860, 2)
CPU times: user 1min 35s, sys: 4.84 s, total: 1min 40s
Wall time: 13.6 s
Umap output:  (1860, 2)
[[0.267004 0.004874 0.329415 1.      ]
 [0.267004 0.004874 0.329415 1.      ]
 [0.267004 0.004874 0.329415 1.      ]
 ...
 [0.916242 0.896091 0.100717 1.      ]
 [0.916242 0.896091 0.100717 1.      ]
 [0.916242 0.896091 0.100717 1.      ]]
p : (1860, 2)


In [ ]:
###################################################
### GENERATE ALL DATA VS. PULLS IN PCA SPACE ######
###################################################

# ************** FIG 2E Convex hull analysis
from Visualize import (get_data_and_triggers, 
                       get_pca_object_and_all_points,
                       project_data_pca,
                       plot_pca_scatter,
                       get_convex_hull,
                       plot_convex_hull,
                       get_sessions)


root_dir = '/media/cat/4TBSSD/yuki/'

animal_ids = ['IA1','IA2','IA3','IJ1',"IJ2","AQ2"]
session_id = 'all'


# 
for animal_id in animal_ids:
    pa = PCA_Analysis()
    pa.root_dir = '/media/cat/4TBSSD/yuki/'
    pa.session_id = 'all'
    pa.animal_id = animal_id

    # 
    pa.n_pca = 10
    pa.sliding_window = 30    # how many frames to take into analysis window
    pa.n_frames = 300         # how many frames back in time to analyze: 

    
    # 
    sessions = get_sessions(pa.root_dir,
                            pa.animal_id,
                            pa.session_id)


    for session in sessions:
        pa.session = session
        print (" processing: ", session)
        fname_out = os.path.join(pa.root_dir,
                                  pa.animal_id,
                                  'tif_files',
                                  pa.session,
                                  pa.session+'_convex_hull.npz')
        if os.path.exists(fname_out):
            print ("     data laready processed")
            print ('')
            continue
            
        fname_triggers = os.path.join(pa.root_dir,
                                      pa.animal_id,
                                      'tif_files',
                                      pa.session,
                                      'blue_light_frame_triggers.npz')

        fname_data = os.path.join(pa.root_dir,
                                     pa.animal_id,
                                     'tif_files',
                                     pa.session,
                                     pa.session+ '_aligned_butterworth_0.1hz_6.0hz.npy')

        pa.X, pa.triggers = get_data_and_triggers(fname_triggers,
                                                  fname_data)
        
        if pa.X is None:
            print ("     missing data/blue light")
            print ('')
            print ('')
            continue
            
        if pa.triggers.shape[0]<5:
            print ("     not enough points : ", pa.triggers.shape )
            print ('')
            print ('')
            continue
            
        # make PCA plots
        pa.pca, pa.all_points = get_pca_object_and_all_points(pa)


        # FIG 2 E top
        pa.p_lever =  project_data_pca(pa)

        # skip plotting option
        # plot_pca_scatter(pa)
         
        # FIG 2E
        pa =  get_convex_hull(pa)

        # skip plotting option
        # plot_convex_hull(pa)
        print ('')


 processing:  IA1pm_Feb1_30Hz
 triggers:  (3,)
 data:  (40000, 128, 128)
(39130, 128, 128)
X:  (39130, 16384)
     not enough points :  (3,)


 processing:  IA1pm_Feb2_30Hz
     data laready processed

 processing:  IA1pm_Feb3_30Hz
     data laready processed

 processing:  IA1pm_Feb4_30Hz
     missing data/blue light


 processing:  IA1pm_Feb5_30Hz
     missing data/blue light


 processing:  IA1pm_Feb9_30Hz
     data laready processed

 processing:  IA1pm_Feb10_30Hz
     data laready processed

 processing:  IA1pm_Feb11_30Hz
     data laready processed

 processing:  IA1pm_Feb12_30Hz
     missing data/blue light


 processing:  IA1pm_Feb15_30Hz
     data laready processed

 processing:  IA1pm_Feb16_30Hz
     data laready processed

 processing:  IA1pm_Feb17_30Hz
     data laready processed

 processing:  IA1pm_Feb18_30Hz
     data laready processed

 processing:  IA1pm_Feb19_30Hz
     data laready processed

 processing:  IA1pm_Feb22_30Hz
 triggers:  (4,)
 data:  (40000, 128, 128)
(3

In [85]:
#############################
##### PLOT CONVEX HULL ######
#############################

# Fig 2 E

fname = '/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb3_30Hz/IA1pm_Feb3_30Hz_convex_hull.npz'
d = np.load(fname, allow_pickle = True)

animal_id = 'IJ1'

pa = PCA_Analysis()
pa.root_dir = '/media/cat/4TBSSD/yuki/'
pa.session_id = 'Feb3_'

pa.n_pca = 10
pa.sliding_window = 30    # how many frames to take into analysis window
pa.n_frames = 300         # how many frames back in time to analyze: 

pa.animal_id = animal_id
pa.ratio_single = d['ratio_single'][::-1]
pa.ratio_cumsum = d['ratio_cumsum'][::-1]
pa.ratio_random_single = d['ratio_random_single'][::-1]
pa.ratio_random_cumulative = d['ratio_random_cumulative'][::-1]


pa.session = get_sessions(pa.root_dir,
                      pa.animal_id,
                      pa.session_id)[0]

print (" processing: ", pa.session)
fname_out = os.path.join(pa.root_dir,
                          pa.animal_id,
                          'tif_files',
                          pa.session,
                          pa.session+'_convex_hull.npz')



pa.X, pa.triggers = get_data_and_triggers(fname_triggers,
                                          fname_data)

if pa.X is None:
    print ("     missing data/blue light")
    print ('')
    print ('')

elif pa.triggers.shape[0]<5:
    print ("     not enough points : ", pa.triggers.shape )
    print ('')
    print ('')
else:
    # make PCA plots
    pa.pca, pa.all_points = get_pca_object_and_all_points(pa)


    # FIG 2 E top
    pa.p_lever =  project_data_pca(pa)


    # scatter plot option
    plot_pca_scatter(pa)

        
# plot_convex_hull(pa)

 processing:  IJ1pm_Feb3_30Hz


NameError: name 'fname_triggers' is not defined

In [5]:
##########################################################
#### GENERATE PCA DISTRIBUTIONS OF ALL BODY MOVEMENTS ####
##########################################################

#  Fig 3E 
pa = PCA_Analysis()
pa.root_dir = '/media/cat/4TBSSD/yuki/'
pa.n_pca = 10
pa.sliding_window = 30    # how many frames to take into analysis window
pa.n_frames = 1         # how many frames back in time to analyze: 

# 
pa.umap_flag = False
pa.convex_hull_flag = False

pa.frame_rate = 30

########################################
animal_ids = ['IA1','IA2','IA3','IJ1',"IJ2","AQ2"]

for animal_id in animal_ids:
    pa.animal_id = animal_id
    session = 'all'

    pa.plot_all = True
    pa.plot_3D = False
    pa.plot_flag = False


    #########################
    # get the correct sesion name:
    sessions = get_sessions(pa.root_dir,
                              pa.animal_id,
                              session)

    print ("sessions: ", sessions)
    res_simplex = pca_scatter_body_movements_fig3(pa, sessions)

print (" COMPLETED: # OF SESSIONS: ", len(res_simplex))

sessions:  ['IA1pm_Feb1_30Hz' 'IA1pm_Feb2_30Hz' 'IA1pm_Feb3_30Hz' 'IA1pm_Feb4_30Hz'
 'IA1pm_Feb5_30Hz' 'IA1pm_Feb9_30Hz' 'IA1pm_Feb10_30Hz' 'IA1pm_Feb11_30Hz'
 'IA1pm_Feb12_30Hz' 'IA1pm_Feb15_30Hz' 'IA1pm_Feb16_30Hz'
 'IA1pm_Feb17_30Hz' 'IA1pm_Feb18_30Hz' 'IA1pm_Feb19_30Hz'
 'IA1pm_Feb22_30Hz' 'IA1pm_Feb23_30Hz' 'IA1pm_Feb24_30Hz'
 'IA1pm_Feb25_30Hz' 'IA1pm_Feb26_30Hz' 'IA1pm_Feb29_30Hz'
 'IA1pm_Mar1_30Hz' 'IA1pm_Mar2_30Hz' 'IA1pm_Mar3_30Hz' 'IA1am_Mar4_30Hz'
 'IA1am_Mar7_30Hz' 'IA1pm_Mar8_30Hz' 'IA1am_Mar9_30Hz' 'IA1am_Mar10_30Hz'
 'IA1am_Mar11_30Hz' 'IA1pm_Mar14_30Hz' 'IA1am_Mar15_30Hz'
 'IA1pm_Mar16_30Hz' 'IA1pm_Mar17_30Hz' 'IA1pm_Mar18_30Hz'
 'IA1pm_Mar21_30Hz' 'IA1pm_Mar22_30Hz' 'IA1pm_Mar23_30Hz'
 'IA1pm_Mar24_30Hz' 'IA1pm_Mar29_30Hz' 'IA1pm_Mar30_30Hz'
 'IA1pm_Mar31_30Hz' 'IA1pm_Apr1_30Hz' 'IA1pm_Apr4_30Hz' 'IA1pm_Apr5_30Hz'
 'IA1pm_Apr6_30Hz' 'IA1pm_Apr7_30Hz' 'IA1pm_Apr11_Day3_30Hz'
 'IA1pm_Apr13_Day5_30Hz' 'IA1pm_Apr15_Day7_30Hz' 'IA1pm_Apr18_Week2_30Hz'
 'IA1pm_Apr19_Week2_3

lever offset:  66
SESSIONS:  ['Feb1_', 'Feb2_', 'Feb3_', 'Feb5_', 'Feb9_', 'Feb10_', 'Feb11_', 'Feb12_', 'Feb15_', 'Feb16_', 'Feb17_', 'Apr21_', 'Apr25_', 'Apr26_', 'Apr27_', 'Apr28_', 'Apr29_', 'May6_', 'May9_', 'May10_', 'May12_', 'May17_', 'May18_', 'May20_']
SHIFT IDS:  ['2', '1', '2', '2', '2', '2', '2', '0', '2', '2', '1', '0', '0', '1', '1', '1', '1', '2', '0', '2', '2', '2', '02', '0']
pa.session_id:  IA1am_May2_Week4_30Hz
pa.session_id 1:  IA1am_May4_Week4_30Hz
session corrected:  IA1am_May4_Week4_30Hz
 No video/3sec movement files...
pa.session_id 1:  IA1am_May5_Week4_30Hz
session corrected:  IA1am_May5_Week4_30Hz
(7,)
['left_paw' 'right_paw' 'nose' 'jaw' 'right_ear' 'tongue' 'lever']
pa.session_id 2:  IA1am_May5_Week4_30Hz
starts:  [ 10.86666667  29.93333333  64.6         90.53333333 170.13333333
 235.         249.2        260.         265.46666667 271.53333333]
lever offset:  94
SESSIONS:  ['Feb1_', 'Feb2_', 'Feb3_', 'Feb5_', 'Feb9_', 'Feb10_', 'Feb11_', 'Feb12_', 'Feb15_',

  0%|          | 0/1 [00:00<?, ?it/s]

 X data using :  30  number of frames  (1299, 491520)
 pca allpoints:  (1299, 10)
1 50 10


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

 pca p_levefr resahped:  (1, 79, 10)
  body movement pa.p_lever projection:  (1, 79, 10)
temp points:  (1298, 2)
temp points:  (79, 2)
plever:  (71, 2)
FNAME LEVER:  /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Apr29_Week8_30Hz/AQ2am_Apr29_Week8_30Hz_all_locs_selected.txt
1 85 10


100%|██████████| 1/1 [00:00<00:00,  2.06it/s]

 pca p_levefr resahped:  (1, 85, 10)
temp points:  (1298, 2)
temp points:  (85, 2)
plever:  (76, 2)
 COMPLETED: # OF SESSIONS:  110



/media/cat/4TBSSD/anaconda3/envs/wide/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


In [89]:
##################################################
### PLOT CONVEX HULLS FOR BODY PARTS VS LEVER ####
##################################################

# # Fig 3E

# def plot_intersection_convex_hulls_lever_vs_bodyparts(res_simplex):
#     from shapely import geometry

#     names = ['leftpaw','rightpaw','nose','jaw','ear','lever']
#     clrs=['black','magenta','blue','green','red','brown']

#     ctr = 0
#     lever_vs_left_paw = []
#     lever_vs_right_paw = []
#     for k in range(len(res_simplex)):
#         pa_simplex = res_simplex[k]
#         if len(pa_simplex)==0:
#             continue

#         if ctr<=45:
#             ax=plt.subplot(5,10,ctr+1)

#         polygons = []
#         for p in range(len(pa_simplex)-1):
#             pol_temp = []
#             if len(pa_simplex[p])>0:
#                 for aa in range(pa_simplex[p].shape[0]):
#                     temp = pa_simplex[p][aa].T

#                     if ctr<=45:
#                         if aa==0:
#                             plt.plot(temp[0],
#                                  temp[1],c=clrs[p],
#                                  label=names[p],
#                                 linewidth=5)
#                         else:
#                             plt.plot(temp[0],
#                                  temp[1],c=clrs[p],
#                                 linewidth=5)


#                     pol_temp.append(temp.T[0])
#                     pol_temp.append(temp.T[1])
#                 pol_temp = np.vstack(pol_temp)
#                 pol_temp = np.unique(pol_temp,axis=0)
#                 pol_temp = convexhull(pol_temp)
#                 polygons.append(pol_temp)
#             else:
#                 polygons.append([])

#         # compute all points outer surface
#         pts_simplex_all_points = pa_simplex[-1]
#         pol_temp =[]
#         for a in range(pts_simplex_all_points.shape[0]):
#             temp = pts_simplex_all_points[a].T
#             if a==0:
#                 plt.plot(temp[0],
#                      temp[1],
#                      c='grey',
#                      label = 'all',
#                     linewidth=5)
#             else:
#                 plt.plot(temp[0],
#                      temp[1],
#                      c='grey',
#                     linewidth=5)

#             pol_temp.append(temp.T[0])
#             pol_temp.append(temp.T[1])

#         pol_temp = np.vstack(pol_temp)
#         pol_temp = np.unique(pol_temp,axis=0)
#         pol_temp = convexhull(pol_temp)
#         polygons_all = pol_temp
#         #

#         # Compute areas of all activity spaces
#         areas = []
#         for p in range(len(polygons)):
#             if len(polygons[p])>0:
#                 polygon1 = geometry.Polygon(polygons[p])
#                 polygon2 = geometry.Polygon(polygons[p])
#                 areas.append(polygon1.intersection(polygon2).area)
#             else:
#                 areas.append(np.nan)
#         print ("all areas: ", areas)
#         polygon1 = geometry.Polygon(polygons_all)
#         polygon2 = geometry.Polygon(polygons_all)
#         area_all = polygon1.intersection(polygon2).area


#         # compute intersections
#         from shapely import geometry, ops
#         for q in range(len(polygons)):
#             for p in range(len(polygons)):
#                 if q==p:
#                     continue

#                 polygon1 = geometry.Polygon(polygons[q])
#                 polygon2 = geometry.Polygon(polygons[p])

#                 intersection = polygon1.intersection(polygon2).area

#                 if q == 0 and p == 5:
#                     if np.isnan(areas[p])==False:
#                         lever_vs_left_paw.append(intersection/areas[q])

#                         print (names[q],names[p],
#                            "% of region: ", round(intersection/areas[q]*100,2), "%",
#                            "% of all space: ", round(intersection/area_all*100,2), "%")
#                     else:
#                         lever_vs_left_paw.append(np.nan)

#                 elif q == 1 and p == 5:
#                     if np.isnan(areas[p])==False:
#                         lever_vs_right_paw.append(intersection/areas[q])
#                         print (names[q],names[p],
#                            "% of region: ", round(intersection/areas[q]*100,2), "%",
#                            "% of all space: ", round(intersection/area_all*100,2), "%")
#                     else:
#                         lever_vs_right_paw.append(np.nan)

#             print ('')
#         plt.xticks([])
#         plt.yticks([])
#         #plt.xlim(left=-150000)
#         #if k==0:
#         #    plt.legend(fontsize=12)
#         plt.title(sessions[k])

#         #
#         ctr+=1

#     return lever_vs_left_paw, lever_vs_right_paw

# 
#
fig=plt.figure(figsize=(5,5))
left, right = plot_intersection_convex_hulls_lever_vs_bodyparts(res_simplex)
plt.suptitle(pa.animal_id)

if False:
    plt.savefig('/home/cat/polygons.png',dpi=300)
    plt.close()
else:
    plt.show()
    
print ("LEFT: ", left)
print ("RIGH:", right)
left = np.array(left)
right = np.array(right)
fig=plt.figure()
idx = np.where(np.isnan(left)==False)[0]
t = np.arange(len(left))[idx]
plt.plot(t, left[idx])


# 
idx = np.where(np.isnan(right)==False)[0]
t = np.arange(len(right))[idx]
plt.plot(t, right[idx])
plt.suptitle(pa.animal_id)
plt.show()

all areas:  [1292221372.5563679, 1302328976.1648726, 1725884964.1035209, 1235154841.371101, 1496037109.1401107, 806673717.3443394]
leftpaw lever % of region:  50.33 % % of all space:  11.42 %

rightpaw lever % of region:  45.75 % % of all space:  10.46 %





all areas:  [4064047838.6372085, 3720101943.1520505, 3597459312.4369164, 3808193508.446695, 3903645151.1560388, 414932385.2819063]
leftpaw lever % of region:  8.19 % % of all space:  3.47 %

rightpaw lever % of region:  9.21 % % of all space:  3.57 %





all areas:  [1731043112.6864903, 1276940984.3328373, 1413305323.8600078, 1587943012.0375938, 1570357844.2156482, 844007687.8063519]
leftpaw lever % of region:  43.65 % % of all space:  14.75 %

rightpaw lever % of region:  36.43 % % of all space:  9.08 %





all areas:  [1801658552.37777, 1616203345.3929052, 1522092963.3031645, 1169244647.3964257, 2011233730.9929595, 1366638900.677206]
leftpaw lever % of region:  61.92 % % of all space:  19.47 %

rightpaw lever % of region:  67.

[0.6881316396361624, 0.4169685497208628, nan, 0.6647743740547633, 0.2710782335175656, 0.4634754402440487, 0.3101919835113635, 0.22196183151399293, 0.36513609049354134, 0.005445332919398934]


In [40]:
print (polygons[0])

[-34157.02254123 -31516.9013172  -25306.25891227 -18400.8155852
 -12575.85030282 -12310.53789496    872.33244449   5669.82166718
  18024.13495447  22738.45637924  41813.2832249   46402.41759252
  50166.65133501  52961.45630633]


In [139]:
import sphviewer as sph
from numpy.ma import masked_array

def myplot(x, y, nb=32, xsize=500, ysize=500):   
    xmin = np.min(x)
    xmax = np.max(x)
    ymin = np.min(y)
    ymax = np.max(y)

    x0 = (xmin+xmax)/2.
    y0 = (ymin+ymax)/2.

    pos = np.zeros([len(x),3])
    pos[:,0] = x
    pos[:,1] = y
    w = np.ones(len(x))

    P = sph.Particles(pos, w, nb=nb)
    S = sph.Scene(P)
    S.update_camera(r='infinity', x=x0, y=y0, z=0, 
                    xsize=xsize, ysize=ysize)
    R = sph.Render(S)
    R.set_logscale()
    img = R.get_image()
    extent = R.get_extent()
    for i, j in zip(range(4), [x0,x0,y0,y0]):
        extent[i] += j
    print (extent)
    
    
    clrs = np.unique(img)
    img_out = np.zeros(img.shape,'float32')
    for k in range(clrs.shape[0]):
        idx = np.where(img==clrs[k])
        print (k)
        img_out[idx]=k
    
    return img_out, extent

# Generate Lever locations
print (pa.p_lever.shape)
x = pa.p_lever.squeeze()[:,0]
y = pa.p_lever.squeeze()[:,1]
x_size = 100
y_size = 100
heatmap_lever, extent_lever = myplot(x,y, nb=4, xsize=x_size, ysize=y_size)

print ("npm ax: ", np.max(heatmap_lever), np.min(heatmap_lever))
print (np.unique(heatmap_lever))

v1_lever = masked_array(heatmap_lever,
                   heatmap_lever>=1)

# Generate left paw locations
print (pa_array[0].shape)
x = pa_array[0].squeeze()[:,0]
y = pa_array[0].squeeze()[:,1]
x_size = 100
y_size = 100
heatmap_leftpaw, extent_left_paw = myplot(x,y, nb=4, xsize=x_size, ysize=y_size)

print ("npm ax: ", np.max(heatmap_leftpaw), np.min(heatmap_leftpaw))
print (np.unique(heatmap_leftpaw))



fig,ax = plt.subplots()

#ax4.imshow(heatmap_64, 
ax1=plt.subplot(121)  
p1 = ax1.imshow(heatmap_lever,
               extent=extent_lever, 
               #origin='lower', 
               aspect='auto',
               cmap='Reds')

plt.xlim(-40000,60000)
plt.ylim(-50000,50000)

#cba = plt.colorbar(pa,shrink=0.25)

# 
ax2=plt.subplot(122)  
# v1_leftpaw = masked_array(heatmap_leftpaw,
#                           heatmap_leftpaw>=1)
p2 = ax2.imshow(heatmap_leftpaw, 
               extent=extent_left_paw, 
               #origin='lower', 
               aspect='auto',
               cmap='Greys')

plt.xlim(-40000,60000)
plt.ylim(-50000,50000)


#cbb = plt.colorbar(pb,shrink=0.25)
#plt.xlabel('Day')
#plt.ylabel('Depth')
#cba.set_label('positive')
#cbb.set_label('negative')
#plt.show()



plt.show()



(1, 31, 10)
[-41973.54    8432.656 -18250.133  34552.848]
0
1
2
npm ax:  2.0 0.0
[0. 1. 2.]
(1, 55, 10)
[-34157.016 265766.25  -61826.69   18024.137]
0
1
2
3
4
npm ax:  4.0 0.0
[0. 1. 2. 3. 4.]


In [37]:
data = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb1_30Hz/IA1pm_Feb1_30Hz_3secNoMove_movements.npz', allow_pickle=True)
features = data['feature_quiescent']
print (features.shape)
labels = data['labels']
print (labels)
temp = np.vstack(features[0])
print (temp[:10])
#plt.plot(features[0])


(7,)
['left_paw' 'right_paw' 'nose' 'jaw' 'right_ear' 'tongue' 'lever']
[[ 0.33333333  6.26666667]
 [ 8.33333333 11.46666667]
 [12.6        33.73333333]
 [37.06666667 51.33333333]
 [55.53333333 62.2       ]
 [63.46666667 75.66666667]
 [75.66666667 81.        ]
 [84.4        89.2       ]
 [89.33333333 93.33333333]
 [95.6        99.86666667]]


In [24]:
#############################################
##### COMPUTE PCA ON EACH SELECTED VALUE ####
#############################################

# FIG 2 E top
pa.p_lever =  project_data_pca(pa)
 
# 
plot_pca_scatter(pa)



100%|██████████| 300/300 [00:04<00:00, 64.71it/s]


 pca p_levefr resahped:  (0, 40, 10)


AttributeError: 'PCA_Analysis' object has no attribute 'all_points'

In [7]:
#####################################################
####### COMPUTE AND PLOT CONVEX HULL DYNAMICS #######
#####################################################

# FIG 2E
#
pa =  get_convex_hull(pa)

#  
plot_convex_hull(pa)

  7%|▋         | 20/300 [00:00<00:01, 190.74it/s]

temp points:  (1314, 3)
# points kept:  (1312,)  of total:  1314
computing convex hull # dim:  3 (1312, 3)
p lever:  (300, 44, 10)


100%|██████████| 300/300 [00:07<00:00, 38.67it/s]


(300, 10)
(300,)


In [64]:
################################################################
################################################################
################################################################

# plot convex hull results


In [59]:
# # UMAP ON DATA
X_U = []
X_U.append(X_lever)
X_U.append(X_random)
X_U.append(X_shifted)
X_U = np.vstack(X_U)
print ("XU_:", X_U.shape)

import umap
umap_3d = umap.UMAP(n_components=3, init='random', random_state=0)

#fit = umap.UMAP()
%time umap_3d.fit(X_U[::10])
u = umap_3d.transform(X_U)

print ("Umap output: ", u.shape)


XU_: (339, 491520)
CPU times: user 1min 6s, sys: 1.09 s, total: 1min 7s
Wall time: 1min 7s
Umap output:  (339, 3)


In [61]:
fig=plt.figure()
#ax = plt.subplot(111)
ax = fig.add_subplot(projection='3d')
ax.scatter(
           u[:p_lever.shape[0],0], 
           u[:p_lever.shape[0],1],
           u[:p_lever.shape[0],2],
            c='red',
            s=100,
            edgecolor = 'black', alpha=1)

ax.scatter(
           u[p_lever.shape[0]:p_lever.shape[0]+p_random.shape[0],0], 
           u[p_lever.shape[0]:p_lever.shape[0]+p_random.shape[0],1], 
           u[p_lever.shape[0]:p_lever.shape[0]+p_random.shape[0],2], 
            c='black',
            s=100,
            edgecolor = 'black', alpha=1)

ax.scatter(
           u[p_lever.shape[0]+p_random.shape[0]:,0], 
           u[p_lever.shape[0]+p_random.shape[0]:,1], 
           u[p_lever.shape[0]+p_random.shape[0]:,2], 
    
            c='pink',
            s=100,
            edgecolor = 'black', alpha=1)
plt.show()

In [69]:
############################################
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

#ax.scatter(xs, ys, zs, marker=m)
ax.scatter(p[:,0], p[:,1], p[:,2],
            c=np.arange(p.shape[0]), 
            s=10, cmap=plt.cm.viridis,
            edgecolor = 'black', alpha=1)
    
ax.scatter(p_lever[:,0], p_lever[:,1],  p_lever[:,2], 
            c='red',
            s=100,
            edgecolor = 'black', alpha=1)

ax.scatter(p_random[:,0], p_random[:,1], p_random[:,2], 
            c='pink',
            s=100,
            edgecolor = 'black', alpha=1)





plt.show()  

In [7]:
############################################
############################################
############################################
fig=plt.figure()
ax=plt.subplot(221)
plt.scatter(p[:,0], p[:,1], 
            c=np.arange(p.shape[0]), 
            s=10, cmap=plt.cm.viridis,
            edgecolor = 'black', alpha=1)
    
plt.scatter(p[triggers,0], p[triggers,1], 
            c='red',
            s=100,
            edgecolor = 'black', alpha=1)
    

ax=plt.subplot(222)
plt.scatter(p2[:,0], p2[:,1], 
            c=np.arange(p2.shape[0]), 
            s=10, cmap=plt.cm.viridis,
            edgecolor = 'black', alpha=1)
    
plt.scatter(p2[triggers,0], p2[triggers,1], 
            c='red',
            s=100,
            edgecolor = 'black', alpha=1)
    
    
ax=plt.subplot(223)
plt.scatter(u[:,0], u[:,1], 
            c=np.arange(u.shape[0]), 
            s=10, cmap=plt.cm.viridis,
            edgecolor = 'black', alpha=1)
    
plt.scatter(u[triggers,0], u[triggers,1], 
            c='red',
            s=100,
            edgecolor = 'black', alpha=1)

plt.show()

In [53]:
#############################################################
#### PLOT DISTRIBUTION OF FIRST SIG DECODING ALL ANIMALS ####
#############################################################

# return all session ids that can be predicted longer than this:
return_ids_threshold = -10

vis.window = 30
vis.lockout_window = 10

# 
vis.min_trials = 20  #min trials used for SVM

vis.lockout=False
# 
vis.plot_first_decoding_time(return_ids_threshold)

# 
if True:
    plt.ylim(-15,0)
    plt.show()
else:
    plt.savefig('/home/cat/first.png',dpi=600)
    plt.close()

all:  ['AQ2am_Feb12_30Hz' 'AQ2am_Feb15_30Hz']
all:  [60 61]
[[-2.8000000000000007, -7.266666666666666, -3.7333333333333343, -2.366666666666667, -3.033333333333335, -3.1999999999999993, -4.033333333333335, -3.466666666666665, -2.2666666666666657, -1.2333333333333343, -3.666666666666668, -2.533333333333335, -2.866666666666667, -2.966666666666665, -2.0666666666666664, -1.0, -2.0666666666666664, -3.2333333333333343, -2.333333333333332, -2.333333333333332, -1.9333333333333336, -1.0, -2.2666666666666657, -1.5666666666666664, -2.4333333333333336, -2.0666666666666664, -1.466666666666665, -1.1999999999999993, -2.0666666666666664, -2.0, -1.6999999999999993], [-1.0, -4.300000000000001, -3.7666666666666657, -1.0, -2.966666666666665, -1.0, -2.166666666666668, -1.5, -1.0, -1.0, -2.1000000000000014, -1.0, -1.533333333333335, -1.2666666666666657, -1.0, -1.0, -4.533333333333335, -4.533333333333335, -3.7666666666666657, -4.266666666666666, -3.7333333333333343, -5.300000000000001, -4.133333333333333, -4.

In [184]:
# 
data = np.load('/home/cat/concatenated_trials.npy')
#data = np.load('/media/cat/4TBSSD/yuki/IJ2/tif_files/IJ2pm_Mar2_30Hz/IJ2pm_Mar2_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy')

fnames = [
'/media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Feb5_30Hz/IA2pm_Feb5_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy',
'/media/cat/4TBSSD/yuki/IJ1/tif_files/IJ1pm_Mar2_30Hz/IJ1pm_Mar2_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy',
'/media/cat/4TBSSD/yuki/IJ2/tif_files/IJ2pm_Mar3_30Hz/IJ2pm_Mar3_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy',
'/media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Apr1_Week4_30Hz/AQ2am_Apr1_Week4_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy',
]

fig=plt.figure(figsize=(15,10))
for ctr,fname in enumerate(fnames):
    data = np.load(fname)
    print ("raw data: ", data.shape)

    data = data.mean(0)[:,0:900].T

    print (data.shape)

    ax=plt.subplot(2,2,ctr+1)
    #plt.scatter(data[:,0],
    #            data[:,1],
    #            c=np.arange(data.shape[0]))
    t = np.arange(data.shape[0])
    for k in range(4):
        plt.plot(t,data[:,k], linewidth=6, label="PCA "+str(k))
    plt.xticks([])
    plt.yticks([])
    if ctr==0:
        plt.legend(fontsize=14)
    plt.xlim(t[0],t[-1])
    plt.ylim(-4,4)
    
if True:
    plt.savefig('/home/cat/pca_vs_time.png',dpi=300)
    plt.close()
else:
    plt.show()


raw data:  (67, 5, 1801)
(900, 5)
raw data:  (61, 7, 1801)
(900, 7)
raw data:  (48, 6, 1801)
(900, 6)
raw data:  (137, 6, 1801)
(900, 6)


In [1]:
from scipy.optimize import curve_fit

def func(x, a, b, c, d):
    return a*np.exp(-c*(x-b))+d

fig=plt.figure()        
plt.scatter(vis.predictions1,
    vis.trials1)
plt.ylim(0,160)
plt.xlim(-10,0)


# fit exponentials
popt, pcov = curve_fit(func,
                       vis.predictions1+30,
                       vis.trials1,
                       [11,1e-6,1e-3,3],
                       maxfev=10000)

print("Popt: ", popt)

x= np.linspace(0,10,1000)
plt.plot(x-10,func(x,*popt))
            
        
print (vis.trials1)
print (vis.predictions1)

NameError: name 'plt' is not defined

In [5]:
#################################################
######### COMPARE RNN VS SVM DECODING ###########
#################################################
# 
fnames_svm = glob2.glob('/media/cat/4TBSSD/yuki/IJ2/SVM_Scores/*.npy')
fnames_rnn= glob2.glob('/media/cat/4TBSSD/yuki/IJ2/RNN_scores/*.npz')

vis.compare_svm_rnn(fnames_svm, fnames_rnn)



In [8]:
#######################################
######## PLOT SVM DECISION TIME #######
#######################################
svm = PredictSVMTime()
svm.main_dir = '/media/cat/4TBSSD/yuki/'

# default params
svm.lockout = False
svm.lockout_window = 10
svm.pca_flag = True
svm.pca_var = 0.95
svm.window = 10

fig = plt.figure()
svm.animal_id = 'AQ2'
svm.session_id = 'all'
svm.get_sessions()

#
ctr=1
for session_id in svm.sessions:
    ax=plt.subplot(8,10,ctr)
    svm.session_id = session_id
    try:
        svm.plot_decision_time(ax)
        plt.title(session_id+", #: "+str(svm.n_trials_both[0]), fontsize=6)
        if ctr!=1:
            plt.yticks([])
        if ctr!=31:
            plt.xticks([])

        ctr+=1

    except:
        pass
plt.suptitle(svm.animal_id,fontsize=20)



Text(0.5, 0.98, 'AQ2')

In [4]:
data = np.load('/media/cat/4TBSSD/yuki/IJ2/SVM_Scores/SVM_Scores_IJ2am_Mar7_30Hz_code_04_trial_ROItimeCourses_10sec_pca_0.95.npy')
print (data.shape)

(601, 10)


In [ ]:

# vis.main_dir = '/media/cat/4TBSSD/yuki/output_3.7-20210310T074752Z-001/output_3.7/no_midline_filter/'

# fname = vis.main_dir+'/'+animal_id+'/SVM_scores_'+animal_id+'_'+str(session)+'.npy'
# vis.load_data(fname)
# vis.plot_decision_choice('black','all - 30frame windows')

# fname = '/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb3_30Hz/analysis/all_accuracy.npy'
# vis.load_data(fname)
# vis.plot_decision_choice('red','all - 1 frame windows')

# fname = vis.main_dir+'/'+animal_id+'/SVM_scores_'+animal_id+'_lockout_'+str(session)+'.npy'
# vis.load_data(fname)
# vis.plot_decision_choice('magenta','lockout')

# # rnn
# fname = vis.main_dir+'/'+animal_id+'/acc_rnn_'+animal_id+'_'+str(session)+'.npy'
# vis.load_data(fname)
# vis.plot_decision_choice('green','rnn-all-denoised')

# fname = vis.main_dir+'/'+animal_id+'/acc_rnn_'+animal_id+'_lockout_'+str(session)+'.npy'
# vis.load_data(fname)
# vis.plot_decision_choice('cyan','rnn-lockout-denoised')


# # #
# # vis.load_data(fname)
# fname = '/media/cat/4TBSSD/yuki/march_7/IA1pm_Feb23_30Hz_code_04_PCA_Denoised/analysis/all_accuracy.npy'
# #fname = vis.main_dir+'/'+animal_id+'/SVM_scores_'+animal_id+'_'+str(session)+'.npy'

# vis.load_data(fname)
# vis.plot_significant('red','all',animal_id, session)
 
# #
# vis.plot_animal_decision_longitudinal('IA1')

# # 
# vis.plot_animal_decision_AUC_longitudinal()

# # # # # 
# lockout=False
# for ctr, name in enumerate(vis.animal_names):
#     ax=plt.subplot(2,3,ctr+1)
#     vis.plot_animal_decision_longitudinal_matrix(name, lockout, ax)
    

#############################################
############## DECISION TIME ################
#############################################

#
# vis.main_dir = '/media/cat/1TB/yuki/yongxu/lever pull/'

# fname = '/media/cat/4TBSSD/yuki/output_3.7-20210310T074752Z-001/output_3.7/no_midline_filter/IA1/conf_10_IA1_0.npy'
# vis.load_data(fname)
# vis.plot_decision_time('red','all')

# animal_name = "IA1"
# vis.plot_decision_time_animal(animal_name)

# animal_name = 'IA1'
# vis.plot_decision_time_animal_matrix(animal_name)

# # 
# vis.filter=False
# vis.plot_decision_time_all_matrix()



#############################################
############## BODY MOVEMENTS ###############
#############################################

# body movement related data
#vis.main_dir = '/media/cat/1TB/yuki/yongxu/body movement/'

# # 
# body_part = 'right_paw'
# fname = vis.main_dir+'/SVM_scores_'+body_part+'_1.npy'
# vis.load_data(fname)
# vis.plot_decision_choice('red','1', body_part)

# fname = vis.main_dir+'/SVM_scores_'+body_part+'_2.npy'
# vis.load_data(fname)
# vis.plot_decision_choice('blue','2', body_part)

# # # 
# fname = vis.main_dir+'/conf_10_left_paw.npy'
# vis.load_data(fname)
# vis.plot_decision_time('red','left')

# fname = vis.main_dir+'/conf_10_right_paw.npy'
# vis.load_data(fname)
# vis.plot_decision_time('blue','right')

# fname = vis.main_dir+'/conf_10_tongue.npy'
# vis.load_data(fname)
# vis.plot_decision_time('green','tongue')


In [7]:
import pickle as pk

file = open('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb3_30Hz/IA1pm_Feb3_30Hz_code_04_lockout_10sec_trial_ROItimeCourses_10sec_pca.pkl', 'rb')

# dump information to that file
pca = pk.load(file)

print (pca.components_.shape)


(1500, 16384)


In [9]:
data_stm = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb3_30Hz/IA1pm_Feb3_30Hz_10.0sec_butterworth_globalAverage_0.1hz_6.0hz_04code_stm.npy')
print (data_stm.shape)

(62, 601, 128, 128)


In [10]:
# 
X = data_stm.reshape(data_stm.shape[0]*data_stm.shape[1],
                     data_stm.shape[2]*data_stm.shape[3])

mu= np.mean(X, axis=0)


NameError: name 'nComp' is not defined

In [11]:
nComp = 9
Xnew = np.dot(pca.transform(X)[:,:nComp],
             pca.components_[:nComp,:])

Xnew+=mu

data_stm_denoised_reshaped = Xnew.reshape(data_stm.shape[0], data_stm.shape[1],
                                          data_stm.shape[2], data_stm.shape[3])


In [17]:
time_filters = pca.transform(X)[:,:nComp]
pca_time_filters_only = time_filters.reshape(data_stm.shape[0], data_stm.shape[1],-1).transpose(0,2,1)
print (pca_time_filters_only.shape)


(62, 9, 601)


(62, 601, 9)


In [16]:
data2 = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb3_30Hz/IA1pm_Feb3_30Hz_code_04_lockout_10sec_trial_ROItimeCourses_10sec.npy')
print (data2.shape)

(40, 35, 601)


In [12]:
print (Xnew.shape)

(37262, 16384)


In [13]:
data_stm_denoised_reshaped.shape

(62, 601, 128, 128)

In [6]:
data = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files.npy')
print (data)

['/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb1_30Hz/IA1pm_Feb1_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb2_30Hz/IA1pm_Feb2_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb3_30Hz/IA1pm_Feb3_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb4_30Hz/IA1pm_Feb4_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb5_30Hz/IA1pm_Feb5_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb9_30Hz/IA1pm_Feb9_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb10_30Hz/IA1pm_Feb10_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb11_30Hz/IA1pm_Feb11_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb12_30Hz/IA1pm_Feb12_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb15_30Hz/IA1pm_Feb15_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb16_30Hz/IA1pm_Feb16_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_f

In [7]:
data = np.load('/media/cat/1TB/yuki/IA1/tif_files/IA1pm_Feb23_30Hz/IA1pm_Feb23_30Hz_code_04_trial_ROItimeCourses_10sec_pca_0.5.npy')
print (data.shape)

(82, 2, 601)


In [9]:
# #
# data1 = np.load('/home/cat/acc_rnn.npy.npz')
# b = data1['b_rnn']
# c = data1['c_s']

# #
# print (b.shape)
# mean = b.mean(1)
# t=np.linspace(-9.5, 9.5, mean.shape[0])
# plt.plot(t, mean, c='black')
# plt.fill_between(t, mean+c, mean-c, color='black', alpha=.2)

# # 
# data2 = np.load('/home/cat/acc_rnn.npy.npz')
# b = data2['b_rnn']
# c = data2['c_s']
# mean = b.mean(1)
# plt.plot(t, mean, 'blue')
# plt.fill_between(t, mean+c, mean-c, color='blue', alpha=.2)


# plt.show()

In [13]:
import numpy as np
import matplotlib.pyplot as plt
data= np.load('/media/cat/1TB/yuki/maskwarp.npy')
print (data.shape)

plt.imshow(data)
plt.show()

(128, 128)


In [138]:
# 
def sum_pixels_in_registered_mask(data, 
                                  maskwarp,
                                  min_pixels=10):

    
    # 
    areas = np.unique(maskwarp)

    
    
    # work in 1D vectors easier to mask
    maskwarp1D = maskwarp.reshape(-1)
    #maskwarp1D = maskwarp

    trial_courses = []
    area_ids = []
    for k in range(data.shape[0]):
        time_courses_local = []

        # convert to 1D vector to mask faster
        data1D = np.float32(data[k].reshape(data[k].shape[0],-1))

        # 
        print (k,"Data1D: ", data1D.shape)
        for id_ in areas:
            idx = np.where(maskwarp1D==id_)[0]
            #idx = np.where(maskwarp1D==id_)

            # only keep areas that have at least 10 pixels
            if idx.shape[0]>min_pixels:
                area_ids.append(id_)
                
                temp = data1D[:,idx]

                if False:
                    # compute DFF
                    F0 = np.nanmean(temp,axis=0)
                    dFF = (data1D[:,idx]-F0)/F0
                else:
                    # skip dFF computation;
                    dFF = temp

                # save average of all pixesl post DFF
                time_courses_local.append(np.nanmean(dFF, axis=1))

            #all_times.append(time_courses_local)
        trial_courses.append(time_courses_local)

    area_ids = np.int32(np.unique(area_ids))
    trial_courses = np.float32(trial_courses)
    print ("# trials, # areas, # times: ", trial_courses.shape)
    print ("area ids: ", area_ids.shape)

    return area_ids, trial_courses

# 
def check_neighbours(mask_roi, idx, p):
    
    x = idx[0][p]
    y = idx[1][p]
    
    sums = 0
    for k in range(-1,2,1):
        for p in range(-1,2,1):
            xx = x+k
            yy = y+p
            if xx < 0 or xx> 127 or yy<0 or yy>127:
                pass
            else:
                sums+=mask_roi[xx,yy]
                if sums>=2:
                    return True
    return False

# 
def remove_single_pixels(maskwarp,
                         min_pixels=10):

    
    
    # load brain mask (i.e. no tissue areas) and apply it to the image mask first
    if True:
        temp = np.int32(np.loadtxt('/media/cat/1TB/yuki/IA1/genericmask.txt'))
        brain_mask = np.ones((128,128),'float32')
        for t in temp:
            brain_mask[t[0],t[1]]=0

        maskwarp = maskwarp*brain_mask
    
    # 
    ids = np.unique(maskwarp)

    ids_selected = []
    masks = []
    mask_b = np.zeros((128,128),'int32')
    for id_ in ids: 
        idx = np.where(maskwarp==id_)
        if idx[0].shape[0]>min_pixels:
            
            # make the mask for the specific ROI
            mask_roi = np.zeros((128,128))
            idx = np.where(maskwarp==id_)

            mask_roi[idx] = 1
            
            #idx_connected = []
            x = []
            y = []
            for p in range(idx[0].shape[0]):
                connected = check_neighbours(mask_roi, idx, p)
                if connected:
                    x.append(idx[0][p])
                    y.append(idx[1][p])
            
            # 
            if len(x)>min_pixels:
                ids_selected.append(int(id_))
                temp = mask_b.copy()
                for p in range(len(x)):
                     temp[x[p],y[p]]=1
                
                masks.append(temp)
                
        
    ids_selected = np.array(ids_selected)
    return ids_selected, masks
# 
maskwarp= np.load('/media/cat/1TB/yuki/maskwarp.npy')

min_pixels = 50
ids, masks = remove_single_pixels(maskwarp,
                                 min_pixels)
print (ids, len(ids))

#ids, courses = sum_pixels_in_registered_mask(data, maskwarp)

# remove certain specific areas:
idx_del =  [0,16,20]
ids = np.delete(ids,0,0)
masks = np.delete(masks,0,0)
print (ids, len(ids))


[  0  15  21  29  36  43  50  57  64  71 136 150 164 186 198 249 255 261
 268 275] 20
[ 15  21  29  36  43  50  57  64  71 136 150 164 186 198 249 255 261 268
 275] 19


In [141]:
# Load names of ROIs
with open('/home/cat/dorsalMaps_name.txt') as f:
    fs = list(f)

fs = np.vstack(fs)
names = []
for name in fs:
    temp = name[0][1:-2]
    names.append(temp)

# load brain mask (i.e. no tissue areas)
temp = np.int32(np.loadtxt('/media/cat/1TB/yuki/IA1/genericmask.txt'))
mask = np.ones((128,128),'float32')
for t in temp:
    mask[t[0],t[1]]=np.nan
    
    
# Plot ROIs
rois = []
final_names = []
for ctr, id_ in enumerate(ids):
    ax=plt.subplot(5,7,ctr+1)
    temp = np.zeros((128,128))+np.nan

    roi = masks[ctr]*mask
    plt.imshow(roi)
    rois.append(roi)
    #
    plt.xticks([])
    plt.yticks([])
    plt.ylabel(str(id_))
    plt.title(names[id_], fontsize=6)
    final_names.append(names[id_])
    
plt.show()
np.savez('/home/cat/rois_'+str(min_pixels)+'.npz',
        rois = rois,
        ids = ids,
        names = final_names)

In [120]:
data = np.load('/home/cat/roi.npz')
rois = data['rois']
plt.imshow(rois[1])

In [4]:
data = np.load('/media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Apr29_Week8_30Hz/AQ2am_Apr29_Week8_30Hz_code_04_random_ROItimeCourses_15sec_pca_0.95.npy')
print (data.shape)
data2 = np.load('/media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Apr29_Week8_30Hz/AQ2am_Apr29_Week8_30Hz_code_04_trial_ROItimeCourses_15sec_pca_0.95.npy')
print (data2.shape)

(81, 5, 901)
(82, 5, 901)


In [20]:
data = np.load('/media/cat/4TBSSD/yuki/AQ2/SVM_Scores/SVM_Scores_AQ2am_Dec29_30Hz_code_04_trial_ROItimeCourses_15sec_pca_0.95.npy')
data.shape

(901, 10)

In [29]:
data = np.load('/media/cat/4TBSSD/yuki/first_decoding_time_all.npz', allow_pickle=True)
session_nos = data['all_session_nos']
session_nos


array([list([]), list([]), list([]), list([]), list([]),
       list([15, 17, 18, 19, 20, 27, 28, 29, 30, 32, 48, 49, 50, 51, 52, 53, 54, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109])],
      dtype=object)

In [21]:
# load rnn
import numpy as np
import matplotlib.pyplot as plt
import os
import glob

# 
rnn_dir = '/mnt/f418adc0-4ff9-410c-93c9-d6d21a1e9c35/yuki/IJ2/temp/rnn/'
svm_dir = '/mnt/f418adc0-4ff9-410c-93c9-d6d21a1e9c35/yuki/IJ2/temp/svm/'

# session:
session = 'Mar9_'

# 
data_rnn = np.load(glob.glob(rnn_dir+'/*'+session+"*.npz")[0])
b_rnn = data_rnn['b_rnn']
print ("rnn data: ", b_rnn.shape)

# load svm
data_svm = np.load(glob.glob(svm_dir+'/*'+session+"*.npy")[0])
data_svm = data_svm[:data_svm.shape[0]//2]
print ("svm data: ", data_svm.shape)

ax=plt.subplot(111)

# plot rnn
std = np.std(b_rnn,1)
mean = np.mean(b_rnn,1)
t=np.arange(b_rnn.shape[0])-9.5
plt.plot(t, mean, color='black')
ax.fill_between(t, mean-std, mean+std, color='black', alpha = 0.2)

# plot svm
std = np.std(data_svm,1)
mean = np.mean(data_svm,1)
t=np.arange(data_svm.shape[0])/30.-10
plt.plot(t, mean, color='blue')
ax.fill_between(t, mean-std, mean+std, color='blue', alpha = 0.2)

plt.xlim(-10,0)
plt.ylim(0.3, 1.0)
plt.show()


rnn data:  (10, 10)
svm data:  (300, 10)


In [113]:
data = np.load('/media/cat/4TBSSD/yuki/IJ2/concatenation_tests/IJ2pm_Feb29_30Hz_random.npy')
print (data.shape)

(93, 6, 900)


In [178]:
animals = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']
root_dir = '/media/cat/4TBSSD/yuki/'
ctr=0
clrs=['black','blue','red','green','magenta','cyan']
for animal in animals:
    fnames = np.load('/media/cat/4TBSSD/yuki/'+animal+'/tif_files.npy')
    n = []
    for fname in fnames:
        session = os.path.split(fname)[1][:-4]
        try:
            data = np.loadtxt(root_dir+animal+'/tif_files/'+session+'/'+session+'_all_locs_selected.txt')
        except:
            continue
        n.append(len(data))

    median = np.mean(n)
    plt.plot([median,median],[0,25],'--',
             linewidth=3,
             c=clrs[ctr])

    #
    total_trials = np.sum(n)
    bins=np.arange(0,210,10)
    y = np.histogram(n, bins)
    plt.plot(y[1][:-1]+5, y[0],
             linewidth=4,
            label=animal+" - "+str(int(median)),
            c=clrs[ctr])
    
    ctr+=1
plt.xlim(bins[0],bins[-1])
plt.legend(fontsize=34)
plt.ylim(0,25)
plt.show()

In [33]:
data = np.load('/media/cat/4TBSSD/yuki/IA1/SVM_Scores/SVM_Scores_IA1pm_Mar2_30Hzcode_04_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz')
acc = data['accuracy']
print (acc.shape)

(1771, 10)


In [185]:
data = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb1_30Hz/blue_light_frame_triggers.npz')

frame_triggers = data['img_frame_triggers']
print (frame_triggers)

[ 7353 22587 31547]


In [186]:
start = data['start_blue']
end = data['end_blue']
print (start,end)



77 39207
